In [22]:
GYRO = False
SEQUENCE_LENGTH = 4
SEQUENCE_OVERLAP = 3
BATCH_SIZE = 10
EPOCHS = 10
MODEL_NAME = f"position_epochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}"

### This code trains a binary classifier which returns "moving" or "static" activity, currently trains on all data


In [23]:
import file_tagger
import sequence_genrator
import tensorflow as tf
import numpy as np
from keras import layers, Sequential, models
from sklearn.model_selection import train_test_split

STATIONARY = {
    "sitting",
    "standing",
    "lying_down_back",
    "lying_down_stomach",
    "lying_down_right",
    "lying_down_left"
}


MOVING = {
    "sitting",
    "standing",
    "lying_down_left",
    "lying_down_right",
    "lying_down_back",
    "lying_down_stomach",
    "walking",
    "ascending_stairs",
    "descending_stairs",
    "shuffle_walking",
    "running",
    "misc_movements"
}

OUTCOMES = ["stationary", "moving"]

DATA_DIRECTORY = "./all_respeck"

In [24]:
# split data into training, dev, and test sets
def train_dev_test_split(data, labels, dev_size, test_size):
    # Split the data into training and temporary (dev + test) sets
    train_data, temp_data, train_labels, temp_labels = train_test_split(data, labels, test_size=(dev_size + test_size))
    
    # Split the temporary data into dev and test sets
    dev_data, test_data, dev_labels, test_labels = train_test_split(temp_data, temp_labels, 
                                                                 test_size=(test_size / (dev_size + test_size)))
    
    return train_data, dev_data, test_data, train_labels, dev_labels, test_labels

In [25]:
# generate training data
def generate_training_data(directory, sequence_length, overlap, gyro = GYRO): # if gyro is false, only accelerometer data is used

    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in csv_dictionary.keys():

        # iterates through each csv file for the activity 
        for csv_file in csv_dictionary[key]:
            if gyro:
                sequences = sequence_genrator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap)
            else:
                sequences = sequence_genrator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap)

            # iterate through each generated sequence
            for sequence in sequences:
                if key.split("&")[0] in STATIONARY:
                    tagged_data.append(("stationary", sequence))
                elif key.split("&")[0] in MOVING:
                    tagged_data.append(("moving", sequence))
                else:
                    print("ERROR: activity not found")
                    return None

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")
    return tagged_data

In [26]:
#CNN MODEL
def train_model_CNN(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    if GYRO:
        width = 6
    else:
        width = 3
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(128, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Dropout(0.5),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    return model

In [27]:



def generate_binary_model(tagged_sequences):
    # Combine all sequences and labels
    sequences = [sequence for _, sequence in tagged_sequences]
    labels = [label for label, _ in tagged_sequences]


    # encode labels to numbers
    sequences = np.array(sequences, dtype=np.float32)
    label_to_index = {label: idx for idx, label in enumerate(OUTCOMES)}
    labels_encoded = [label_to_index[label] for label in labels]
    labels_encoded = np.array(labels_encoded)

    train_data, dev_data, test_data, train_labels, dev_labels, test_labels = train_dev_test_split(sequences, labels_encoded, dev_size=0.1, test_size=0.1) #10% dev, 10% test


    # train and save model (CHOOSE BETWEEN CNN AND LSTM)
    model = train_model_CNN(train_data, train_labels, OUTCOMES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(dev_data, dev_labels)) #batch_size, epochs


    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(test_data, test_labels)
    print (f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")
    return model

In [28]:
tagged_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, GYRO)
model = generate_binary_model(tagged_sequences)

# Save the trained model
#model.save(f"models/Task1/{MODEL_NAME}_{test_accuracy}.keras")

there are 54563 tagged sequences in the dataset
Epoch 1/10
4365/4365 [==============================] - 13s 3ms/step - loss: 0.0562 - accuracy: 0.9811 - val_loss: 0.0230 - val_accuracy: 0.9930
Epoch 2/10
4365/4365 [==============================] - 12s 3ms/step - loss: 0.0281 - accuracy: 0.9909 - val_loss: 0.0237 - val_accuracy: 0.9930
Epoch 3/10
4365/4365 [==============================] - 12s 3ms/step - loss: 0.0202 - accuracy: 0.9931 - val_loss: 0.0115 - val_accuracy: 0.9967
Epoch 4/10
4365/4365 [==============================] - 12s 3ms/step - loss: 0.0205 - accuracy: 0.9934 - val_loss: 0.0086 - val_accuracy: 0.9973
Epoch 5/10
4365/4365 [==============================] - 12s 3ms/step - loss: 0.0162 - accuracy: 0.9953 - val_loss: 0.0105 - val_accuracy: 0.9962
Epoch 6/10
4365/4365 [==============================] - 12s 3ms/step - loss: 0.0137 - accuracy: 0.9955 - val_loss: 0.0095 - val_accuracy: 0.9969
Epoch 7/10
4365/4365 [==============================] - 13s 3ms/step - loss: 0.014